### Run times
We can use a simple multivariate normal example to compare the run times for different dimension summary statistics and different dimension of parameters, because the run time of the simulator is negligable.

## Imports

In [197]:
using Pkg
using SyntheticLikelihood
using Distributions
using BenchmarkTools
using DataFrames
using CSV

## How it scales with summary statistics
Keeping the parameters 10 dimensional, we can draw more random samples to get more summary statistics.

In [198]:
d = 10
prior = Prior(fill(Uniform(-1,1), d))
draws = [1, 2, 3, 4, 5];

### Loop through and get results

In [199]:
n_sum_stats = []
rula_times = []
rwm_times = []

for n_draws in draws
    
    # Define the simulator
    function simulator(θ::Vector{Float64})
        @assert length(θ) == 10
        d = MvNormal(θ, sqrt(0.1))
        s = rand(d, n_draws)
        reshape(s, length(s))
    end
    
    θ_true = sample_θ(prior)
    s_true = simulator(θ_true)
    
    local_posterior = LocalPosterior(; simulator, s_true, n_sim = 1000, prior)
    basic_posterior = BasicPosterior(; simulator, s_true, n_sim = 1000, prior)

    rula = RiemannianULA(0.5)
    rwm = RWMetropolis(MvNormal(fill(sqrt(0.1), d)))
    
    # Warm up samplers so compiled
    run_sampler!(rula, local_posterior; init_θ = θ_true, n_steps = 3)  
    run_sampler!(rwm, basic_posterior; init_θ = θ_true, n_steps = 3)  
    
    append!(n_sum_stats, d*n_draws)
    append!(rula_times, mean([@elapsed run_sampler!(rula, local_posterior; init_θ = θ_true, n_steps = 100) for rep in 1:3]))
    append!(rwm_times, mean([@elapsed run_sampler!(rwm, basic_posterior; init_θ = θ_true, n_steps = 100) for rep in 1:3]))
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:28
Progress: 100%|█████████████████████████████████████████| Time: 0:00:24
Progress: 100%|█████████████████████████████████████████| Time: 0:00:21
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:35
Progress: 100%|█████████████████████████████████████████| Time: 0:00:38
Progress: 100%|█████████████████████████████████████████| Time: 0:00:39
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:56
Progress: 100%|█████████████████████████████████████████| Time: 

In [200]:
sum_stat_times_df = DataFrame(n_sum_stats=n_sum_stats, rula_time_per_step = rula_times./100, rwm_time_per_step=rwm_times./100)

,n_sum_stats,rula_time_per_step,rwm_time_per_step
,Any,Float64,Float64
1,10,0.251984,0.000364797
2,20,0.38306,0.00130184
3,30,0.587958,0.00140777
4,40,0.679676,0.00810087
5,50,0.834216,0.0136643


In [201]:
CSV.write("results/mvn_times_sum_stats.csv", sum_stat_times_df)

"results/mvn_times_sum_stats.csv"

## Scaling with the number of parameters

In [202]:
ns = 10
dimensions = [10, 20, 30, 40];

In [208]:
function chunkify(A, n)
    [A[i:min(i + n - 1, end)] for i in 1:n:length(A)]
end

chunkify (generic function with 1 method)

In [209]:
n_parameters = []
rula_times = []
rwm_times = []

for nθ in dimensions
    
    prior = Prior(fill(Normal(), nθ))
    
    # Define the simulator
    function simulator(θ::Vector{Float64})
        @assert length(θ) == nθ
        d = MvNormal(θ, sqrt(0.1))
        s = rand(d)
        
        s_chunks = chunkify(s, Int(nθ/10))
        s = mean.(s_chunks)
        s
    end
    
    θ_true = sample_θ(prior)
    s_true = simulator(θ_true)
    
    local_posterior = LocalPosterior(; simulator, s_true, n_sim = 1000, prior)
    basic_posterior = BasicPosterior(;simulator, s_true, n_sim = 1000, prior)
    
    rula = RiemannianULA(0.5)
    rwm = RWMetropolis(MvNormal(fill(sqrt(0.1), nθ)))
    
    # Warm up samplers so compiled
    run_sampler!(rula, local_posterior; init_θ = θ_true, n_steps = 3)  
    run_sampler!(rwm, basic_posterior; init_θ = θ_true, n_steps = 3)  
    
    append!(n_parameters, nθ)
    append!(rula_times, mean([@elapsed run_sampler!(rula, local_posterior; init_θ = θ_true, n_steps = 100) for rep in 1:3]))
    append!(rwm_times, mean([@elapsed run_sampler!(rwm, basic_posterior; init_θ = θ_true, n_steps = 100) for rep in 1:3]))
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:10
Progress: 100%|█████████████████████████████████████████| Time: 0:00:18
Progress: 100%|█████████████████████████████████████████| Time: 0:00:13
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:31
Progress: 100%|█████████████████████████████████████████| Time: 0:00:49
Progress: 100%|█████████████████████████████████████████| Time: 0:00:25
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 

In [210]:
θ_times_df = DataFrame(n_parameters=n_parameters, rula_time_per_step = rula_times./100, rwm_time_per_step=rwm_times./100)

,n_parameters,rula_time_per_step,rwm_time_per_step
,Any,Float64,Float64
1,10,0.139742,0.00360331
2,20,0.358513,0.00289465
3,30,1.52501,0.00437852
4,40,2.07087,0.00356408


In [211]:
CSV.write("results/mvn_times_parameters.csv", θ_times_df)

"results/mvn_times_parameters.csv"